# Experiment with using Haar Cascades to identify the dots

Gary Bishop July 2018

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# %config InlineBackend.print_figure_kwargs={'bbox_inches':None}
import cv2
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl

mpl.rcParams['figure.dpi']= 300

vid = '/home/gb/Dropbox/Karen and Gary Shared Files/Videos & Transcripts/MSB/MSB_Video 1 (09-30-17).mp4'
vc = cv2.VideoCapture(vid)

def show(im, **kwargs):
    '''Show images actual size unless it is tiny
    
    I'm assuming they are in LAB float32 if the rank is 3
    
    '''
    height, width = im.shape[:2]
    if height > 50 and width > 50:
        dpi = 100
        margin= 50
        figsize=((width+2*margin)/dpi, (height+2*margin)/dpi) # inches
        left = margin/dpi/figsize[0] #axes ratio
        bottom = margin/dpi/figsize[1]

        fig = plt.figure(figsize=figsize, dpi=dpi)
        fig.subplots_adjust(left=left, bottom=bottom, right=1.-left, top=1.-bottom)
    else:
        plt.figure()
    
    args = dict(kwargs)
    if 'title' in args:
        del args['title']
    
    if len(im.shape) == 3:
        im = cv2.cvtColor(im, cv2.COLOR_LAB2RGB)
    elif len(im.shape) == 2:
        args['cmap'] = 'gray'                  

    plt.imshow(im, **args)
    if 'title' in kwargs:
        plt.title(kwargs['title'])

def grabFrame(fn):
    vc.set(cv2.CAP_PROP_POS_FRAMES, fn)
    rval, im = vc.read()
    im = cv2.cvtColor(im.astype(np.float32)/255.0, cv2.COLOR_BGR2LAB)
    return im
show(grabFrame(100))

In [ ]:
def isBlue(im):
    mblue = np.array([ 60.4 , -12.2, -35.7 ], dtype=np.float32)
    sblue = np.array([ 4.1, 3.2, 8.5], dtype=np.float32)
    d2 = np.sum((im - mblue)**2 / sblue**2, axis=2)
    return np.exp(-d2 / 20)


In [ ]:
cascade = cv2.CascadeClassifier('output/cascade.xml')
frame = grabFrame(1700)
#show(frame)
bim = (255 * isBlue(frame)).astype(np.uint8)
# bim[bim < 0.8] = 0
show(bim)
dots = cascade.detectMultiScale(bim, 1.001)
for x, y, w, h in dots:
    cv2.rectangle(frame, (x, y), (x+w, y+h), (0,0,0), 1)
show(frame)

## Make more negatives

In [ ]:
import os
fp = open('negative.dat', 'at')
offset = len(os.listdir('negative'))
for i, (x, y, w, h) in enumerate(dots):
    if y+h > 300:
        continue
    pixels = frame[y:y+h,x:x+w]
    bim = isBlue(pixels)
    bim = (255 * isBlue(pixels)).astype(np.uint8)
    fname = 'negative/dot{:03d}.png'.format(i+offset)
    cv2.imwrite(fname, bim)
    print(fname, file=fp)